In [3]:
# default setting
from tensorflow import keras

In [ ]:
# AMD setting
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
os.environ["PLAIDML_EXPERIMENTAL"] = "1"
os.environ["PLAIDML_DEVICE_IDS"] = "opencl_amd_ellesmere.0"
import keras

In [1]:
import Generator

In [2]:
Generator.generate(100) #100개씩 만듦

In [5]:
#fade : 5프레임 - 0.5초 안에 

In [6]:
import FFmpegDecoder

In [7]:
FFmpegDecoder.video_to_image("sample.mp4", "sample_images", 20) #초당 20장 

In [4]:
import TwoImageConcat

In [ ]:
# Make new model with random weights
model = TwoImageConcat.make_model(keras)
TwoImageConcat.compile_model(model)

# Train model
TwoImageConcat.train_model(model, epochs=2)

# Save model
TwoImageConcat.save_model("model.h5", model)

In [7]:
# Make new model with saved weights
model = TwoImageConcat.load_model(keras, "model.h5")
TwoImageConcat.compile_model(model)

# Check validation
(_, data_gen) = TwoImageConcat.get_data_gen(keras, 100)
TwoImageConcat.check_validation(model, data_gen)

Found 0 images belonging to 0 classes.
Found 400 images belonging to 4 classes.
              precision    recall  f1-score   support

         0.0       0.94      0.62      0.74        26
         1.0       0.80      1.00      0.89        20
         2.0       0.95      0.70      0.81        27
         3.0       0.66      0.93      0.77        27

    accuracy                           0.80       100
   macro avg       0.84      0.81      0.80       100
weighted avg       0.84      0.80      0.80       100

[[16  4  0  6]
 [ 0 20  0  0]
 [ 1  0 19  7]
 [ 0  1  1 25]]
